In [2]:
# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.backtester.base.base import rsi_below_entries, plot_trades_all_info
from quantfreedom.backtester.nb.simulate import simulate_up_to_6
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv('../30min.csv', index_col='time')

entries = rsi_below_entries(
    timeperiods=[15],
    below_ranges=[40],
    prices=prices.close
)


In [4]:
final_array, order_records = simulate_up_to_6(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    gains_pct_filter=40,
    # sl_pcts=[2,3,4],
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2,5,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,4,1),
    tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)


In [11]:
df_or = pd.DataFrame(order_records)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.head(20)

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
0,0,0,31,471.97,"10,850.00","10,850.00",NaN,LongEntry,NaN,"1,000.00",NaN,"10,633.00","11,553.11"
1,1,0,32,471.97,"10,725.00","10,787.14",NaN,LongEntry,NaN,"1,000.00",NaN,"10,571.40","11,486.18"
2,2,0,32,943.93,"10,571.40","10,787.14",1.12,LongTSL,-20.00,980.00,NaN,"10,571.40","11,486.18"
3,3,0,33,462.53,"10,577.50","10,577.50",NaN,LongEntry,NaN,980.00,NaN,"10,365.95","11,262.95"
4,4,0,34,462.53,"10,523.00","10,550.18",NaN,LongEntry,NaN,980.00,NaN,"10,339.18","11,233.86"
5,5,0,35,462.53,"10,516.00","10,538.76",NaN,LongEntry,NaN,980.00,NaN,"10,327.99","11,221.70"
6,6,0,36,462.53,"10,489.00","10,526.28",NaN,LongEntry,NaN,980.00,NaN,"10,315.75","11,208.41"
7,7,0,183,NaN,NaN,"10,526.28",NaN,MovedTSL,NaN,980.00,NaN,"10,547.46","11,208.41"
8,8,0,184,NaN,NaN,"10,526.28",NaN,MovedTSL,NaN,980.00,NaN,"10,557.36","11,208.41"
9,9,0,193,NaN,NaN,"10,526.28",NaN,MovedTSL,NaN,980.00,NaN,"10,590.03","11,208.41"


In [6]:
df_cart = pd.DataFrame(final_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

,0,1,2
total_trades,195.00,118.00,77.00
gains_pct,-22.95,16.51,100.39
win_rate,37.95,45.76,51.95
to_the_upside,0.67,0.54,0.29
total_pnl,-229.47,165.12,"1,003.88"
ending_eq,770.53,"1,165.12","2,003.88"
settings_id,0,1,2
max_equity_risk_pct,4.00,4.00,4.00
risk_rewards,3.00,5.00,6.00
size_pct,1.00,1.00,1.00


In [7]:
plot_trades_all_info(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_records[:30],
)

In [ ]:
clear_cache()